<a href="https://colab.research.google.com/github/Gowri0109/-Image-Processing-for-Brain-Tumor-Classification-and-detection/blob/new/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Gowri0109/-Image-Processing-for-Brain-Tumor-Classification-and-detection.git

Cloning into '-Image-Processing-for-Brain-Tumor-Classification-and-detection'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 128 (delta 72), reused 81 (delta 32), pack-reused 0
Receiving objects: 100% (128/128), 980.19 KiB | 7.21 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog

# Define the paths to the directories for each set
train_dir = "/content/training_set"
# val_dir = "D:\\master_thesis\\datasets\\data_split\\validation_set"
# test_dir = "D:\\master_thesis\\datasets\\data_split\\test_set"

# Define the output directories for saving the feature vectors
output_dir_train = "/content/output_features"
# output_dir_val = "D:\\master_thesis\\datasets\\hog_feature\\val"
# output_dir_test = "D:\\master_thesis\\datasets\\hog_feature\\test"

# Define the HOG parameters
cell_size = (8, 8)
block_size = (2, 2)
num_bins = 9

# Function to extract HOG features from a set of images
def extract_hog_features(image_dir, output_dir):
    features = []
    labels = []

    # Loop over each subdirectory (class) in the image directory
    for subdir in os.listdir(image_dir):
        sub_dir_path = os.path.join(image_dir, subdir)
        
        # Loop over each image file in the subdirectory
        for file in os.listdir(sub_dir_path):
            image_path = os.path.join(sub_dir_path, file)
            
            # Read the image
            image = cv2.imread(image_path)
            
            # Convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Calculate the HOG features
            hog_features = hog(gray, orientations=num_bins, pixels_per_cell=cell_size, cells_per_block=block_size)
            
            # Append the features and label
            features.append(hog_features)
            labels.append(subdir)
    
    # Convert the feature and label lists to numpy arrays
    features = np.array(features)
    labels = np.array(labels)
    
    # Save the feature vectors and labels to files
    np.save(os.path.join(output_dir, "hog_features.npy"), features)
    np.save(os.path.join(output_dir, "labels.npy"), labels)

# # Extract HOG features for the training set and save the features in the output directory
extract_hog_features(train_dir, output_dir_train)

# # Extract HOG features for the validation set and save the features in the output directory
# extract_hog_features(val_dir, output_dir_val)

# Extract HOG features for the test set and save the features in the output directory
# extract_hog_features(test_dir, output_dir_test)


In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import cv2

input_folder = "D:\\master_thesis\\datasets\\data_split\\test_set"
output_folder = "D:\\master_thesis\\datasets\\test_model"

# Iterate over the folders in the input folder (assuming each folder represents a tumor type)
for tumor_folder in os.listdir(input_folder):
    tumor_folder_path = os.path.join(input_folder, tumor_folder)
    if not os.path.isdir(tumor_folder_path):
        continue

    print("Processing tumor folder:", tumor_folder)

    # Load the features from the .npy file
    feature_folder = os.path.join(tumor_folder_path, "hog_feature")  # Adjust folder name as per your case
    feature_file_path = os.path.join(feature_folder, "D:\\master_thesis\\datasets\\hog_feature\\test\\hog_features.npy")  # Adjust file name as per your case

    features = np.load(feature_file_path, allow_pickle=True)
    
    # Load the labels from the .npy file
    label_file_path = os.path.join(feature_folder, "D:\\master_thesis\\datasets\\hog_feature\\test\\labels.npy")  # Adjust file name as per your case
    labels = np.load(label_file_path, allow_pickle=True)
    
    # Check if features and labels arrays are empty
    if features.size == 0 or labels.size == 0:
        print("No feature files found for tumor folder:", tumor_folder)
        continue

    # Reshape the feature data if needed (uncomment the line below and modify the shape)
    # features = features.reshape(features.shape[0], -1)

    # Train a Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    rf_classifier.fit(features, labels)

    # Save the trained model
    model_file_path = os.path.join(output_folder, tumor_folder + ".joblib")
    joblib.dump(rf_classifier, model_file_path)

    # Predict on the training data
    predictions = rf_classifier.predict(features)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)
    print("Accuracy for tumor folder", tumor_folder, ":", accuracy)

    print("Completed processing tumor folder:", tumor_folder)

print("Model training completed.")